In [79]:
import numpy as np
import pandas as pd
pd.set_option('max_rows',38)
pd.set_option('max_columns',25)
import time
import pickle
import myslack

# model
from sklearn.externals import joblib
from scipy.sparse import csr_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
import xgboost
import lightgbm
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import validation_curve
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import classification_report

# Data Check

#### - train.csv

In [3]:
%%time
train = pickle.load(open("./data/final_train_Ver3.pkl", "rb"))
print("test data shape: "+str(train.shape))

test data shape: (95674, 5494)
CPU times: user 200 ms, sys: 626 ms, total: 825 ms
Wall time: 839 ms


#### - test.csv

In [4]:
# %%time
test = pickle.load(open("./data/final_test_Ver3.pkl", "rb"))
print("test data shape: "+str(test.shape))

test data shape: (95674, 5493)


---

# Ver3 : use Dmatrix

# Modeling 6 - LightGBM

In [5]:
# in test, not in train
not_in_train = [i for i in test.columns if i not in train.columns[1:]]

In [6]:
# in train, not in test
not_in_test = [i for i in train.columns[1:] if i not in test.columns]

In [7]:
len(not_in_train)

328

In [8]:
len(not_in_test)

328

In [9]:
train.drop(columns=not_in_test,inplace=True)

In [10]:
train.shape

(95674, 5166)

In [11]:
test.drop(columns=not_in_train,inplace=True)

In [12]:
test.shape

(95674, 5165)

### Train_X, Train_y

In [13]:
X = train.drop(columns='TripType')

In [14]:
y = train['TripType']

In [15]:
from sklearn.preprocessing import LabelEncoder
label_enc = LabelEncoder().fit(y)
y_labeled = label_enc.transform(y)

In [16]:
len(pd.Series(y_labeled).unique())

38

In [108]:
X_train, X_test, y_train, y_test = train_test_split(X, y_labeled, test_size=0.1, random_state=0)

In [109]:
X_train = csr_matrix(X_train.values)
X_test = csr_matrix(X_test.values)

In [152]:
# 1500, 0.02
%%time

lightgbm_model = lightgbm.train(params = params,
                                train_set = dtrain, 
                                num_boost_round = num_boost_round,
                                valid_sets = dtest,
                                early_stopping_rounds=20,
                                )

[1]	valid_0's multi_logloss: 3.4387
Training until validation scores don't improve for 20 rounds.
[2]	valid_0's multi_logloss: 3.28717
[3]	valid_0's multi_logloss: 3.16212
[4]	valid_0's multi_logloss: 3.05413
[5]	valid_0's multi_logloss: 2.96012
[6]	valid_0's multi_logloss: 2.87578
[7]	valid_0's multi_logloss: 2.7996
[8]	valid_0's multi_logloss: 2.72978
[9]	valid_0's multi_logloss: 2.66621
[10]	valid_0's multi_logloss: 2.60696
[11]	valid_0's multi_logloss: 2.55164
[12]	valid_0's multi_logloss: 2.50001
[13]	valid_0's multi_logloss: 2.45157
[14]	valid_0's multi_logloss: 2.40608
[15]	valid_0's multi_logloss: 2.36247
[16]	valid_0's multi_logloss: 2.32138
[17]	valid_0's multi_logloss: 2.28246
[18]	valid_0's multi_logloss: 2.2451
[19]	valid_0's multi_logloss: 2.20932
[20]	valid_0's multi_logloss: 2.17505
[21]	valid_0's multi_logloss: 2.14234
[22]	valid_0's multi_logloss: 2.11079
[23]	valid_0's multi_logloss: 2.08069
[24]	valid_0's multi_logloss: 2.05167
[25]	valid_0's multi_logloss: 2.02365


[211]	valid_0's multi_logloss: 0.830115
[212]	valid_0's multi_logloss: 0.828909
[213]	valid_0's multi_logloss: 0.827706
[214]	valid_0's multi_logloss: 0.826465
[215]	valid_0's multi_logloss: 0.825256
[216]	valid_0's multi_logloss: 0.824066
[217]	valid_0's multi_logloss: 0.8229
[218]	valid_0's multi_logloss: 0.821723
[219]	valid_0's multi_logloss: 0.820623
[220]	valid_0's multi_logloss: 0.819467
[221]	valid_0's multi_logloss: 0.818277
[222]	valid_0's multi_logloss: 0.817161
[223]	valid_0's multi_logloss: 0.816087
[224]	valid_0's multi_logloss: 0.814979
[225]	valid_0's multi_logloss: 0.813941
[226]	valid_0's multi_logloss: 0.812888
[227]	valid_0's multi_logloss: 0.811834
[228]	valid_0's multi_logloss: 0.810763
[229]	valid_0's multi_logloss: 0.809757
[230]	valid_0's multi_logloss: 0.808716
[231]	valid_0's multi_logloss: 0.807679
[232]	valid_0's multi_logloss: 0.806672
[233]	valid_0's multi_logloss: 0.805604
[234]	valid_0's multi_logloss: 0.804617
[235]	valid_0's multi_logloss: 0.803588
[2

[417]	valid_0's multi_logloss: 0.70906
[418]	valid_0's multi_logloss: 0.708777
[419]	valid_0's multi_logloss: 0.708527
[420]	valid_0's multi_logloss: 0.708264
[421]	valid_0's multi_logloss: 0.70803
[422]	valid_0's multi_logloss: 0.707796
[423]	valid_0's multi_logloss: 0.707548
[424]	valid_0's multi_logloss: 0.707289
[425]	valid_0's multi_logloss: 0.707062
[426]	valid_0's multi_logloss: 0.706772
[427]	valid_0's multi_logloss: 0.7065
[428]	valid_0's multi_logloss: 0.706256
[429]	valid_0's multi_logloss: 0.706017
[430]	valid_0's multi_logloss: 0.705745
[431]	valid_0's multi_logloss: 0.705526
[432]	valid_0's multi_logloss: 0.705241
[433]	valid_0's multi_logloss: 0.704982
[434]	valid_0's multi_logloss: 0.704745
[435]	valid_0's multi_logloss: 0.704511
[436]	valid_0's multi_logloss: 0.704269
[437]	valid_0's multi_logloss: 0.704023
[438]	valid_0's multi_logloss: 0.703784
[439]	valid_0's multi_logloss: 0.703541
[440]	valid_0's multi_logloss: 0.70332
[441]	valid_0's multi_logloss: 0.703055
[442]

[623]	valid_0's multi_logloss: 0.67434
[624]	valid_0's multi_logloss: 0.674218
[625]	valid_0's multi_logloss: 0.674088
[626]	valid_0's multi_logloss: 0.673969
[627]	valid_0's multi_logloss: 0.673845
[628]	valid_0's multi_logloss: 0.673717
[629]	valid_0's multi_logloss: 0.673597
[630]	valid_0's multi_logloss: 0.673462
[631]	valid_0's multi_logloss: 0.67336
[632]	valid_0's multi_logloss: 0.67325
[633]	valid_0's multi_logloss: 0.673148
[634]	valid_0's multi_logloss: 0.673067
[635]	valid_0's multi_logloss: 0.672952
[636]	valid_0's multi_logloss: 0.672876
[637]	valid_0's multi_logloss: 0.67277
[638]	valid_0's multi_logloss: 0.672674
[639]	valid_0's multi_logloss: 0.672567
[640]	valid_0's multi_logloss: 0.672453
[641]	valid_0's multi_logloss: 0.672368
[642]	valid_0's multi_logloss: 0.672276
[643]	valid_0's multi_logloss: 0.672153
[644]	valid_0's multi_logloss: 0.672034
[645]	valid_0's multi_logloss: 0.671912
[646]	valid_0's multi_logloss: 0.671824
[647]	valid_0's multi_logloss: 0.671738
[648

[829]	valid_0's multi_logloss: 0.658902
[830]	valid_0's multi_logloss: 0.658866
[831]	valid_0's multi_logloss: 0.658806
[832]	valid_0's multi_logloss: 0.658758
[833]	valid_0's multi_logloss: 0.658679
[834]	valid_0's multi_logloss: 0.658626
[835]	valid_0's multi_logloss: 0.658559
[836]	valid_0's multi_logloss: 0.658552
[837]	valid_0's multi_logloss: 0.658484
[838]	valid_0's multi_logloss: 0.658418
[839]	valid_0's multi_logloss: 0.658352
[840]	valid_0's multi_logloss: 0.658317
[841]	valid_0's multi_logloss: 0.658284
[842]	valid_0's multi_logloss: 0.658226
[843]	valid_0's multi_logloss: 0.658161
[844]	valid_0's multi_logloss: 0.658134
[845]	valid_0's multi_logloss: 0.658043
[846]	valid_0's multi_logloss: 0.657989
[847]	valid_0's multi_logloss: 0.657948
[848]	valid_0's multi_logloss: 0.657878
[849]	valid_0's multi_logloss: 0.657675
[850]	valid_0's multi_logloss: 0.657602
[851]	valid_0's multi_logloss: 0.657526
[852]	valid_0's multi_logloss: 0.657442
[853]	valid_0's multi_logloss: 0.657395


[1034]	valid_0's multi_logloss: 0.651187
[1035]	valid_0's multi_logloss: 0.651155
[1036]	valid_0's multi_logloss: 0.651141
[1037]	valid_0's multi_logloss: 0.651107
[1038]	valid_0's multi_logloss: 0.651091
[1039]	valid_0's multi_logloss: 0.651063
[1040]	valid_0's multi_logloss: 0.651049
[1041]	valid_0's multi_logloss: 0.651028
[1042]	valid_0's multi_logloss: 0.651
[1043]	valid_0's multi_logloss: 0.651012
[1044]	valid_0's multi_logloss: 0.650978
[1045]	valid_0's multi_logloss: 0.650959
[1046]	valid_0's multi_logloss: 0.650937
[1047]	valid_0's multi_logloss: 0.65089
[1048]	valid_0's multi_logloss: 0.650859
[1049]	valid_0's multi_logloss: 0.65082
[1050]	valid_0's multi_logloss: 0.650816
[1051]	valid_0's multi_logloss: 0.65075
[1052]	valid_0's multi_logloss: 0.650739
[1053]	valid_0's multi_logloss: 0.650746
[1054]	valid_0's multi_logloss: 0.650755
[1055]	valid_0's multi_logloss: 0.650735
[1056]	valid_0's multi_logloss: 0.650739
[1057]	valid_0's multi_logloss: 0.650734
[1058]	valid_0's multi

[1235]	valid_0's multi_logloss: 0.648603
[1236]	valid_0's multi_logloss: 0.648598
[1237]	valid_0's multi_logloss: 0.648597
[1238]	valid_0's multi_logloss: 0.648583
[1239]	valid_0's multi_logloss: 0.648562
[1240]	valid_0's multi_logloss: 0.648555
[1241]	valid_0's multi_logloss: 0.648565
[1242]	valid_0's multi_logloss: 0.648573
[1243]	valid_0's multi_logloss: 0.648563
[1244]	valid_0's multi_logloss: 0.648567
[1245]	valid_0's multi_logloss: 0.648569
[1246]	valid_0's multi_logloss: 0.648548
[1247]	valid_0's multi_logloss: 0.648535
[1248]	valid_0's multi_logloss: 0.648541
[1249]	valid_0's multi_logloss: 0.648532
[1250]	valid_0's multi_logloss: 0.64854
[1251]	valid_0's multi_logloss: 0.648516
[1252]	valid_0's multi_logloss: 0.648523
[1253]	valid_0's multi_logloss: 0.648512
[1254]	valid_0's multi_logloss: 0.64849
[1255]	valid_0's multi_logloss: 0.648458
[1256]	valid_0's multi_logloss: 0.648469
[1257]	valid_0's multi_logloss: 0.648461
[1258]	valid_0's multi_logloss: 0.648471
[1259]	valid_0's m

In [194]:
myslack.send_slack('Lightgbm model training finish!, best_score : {}'.format(lightgbm_model.best_score))

<Response [200]>


In [200]:
lightgbm_model.best_iteration = 3487

In [201]:
lightgbm_model.best_score

defaultdict(dict, {'valid_0': {'multi_logloss': 0.6500478227461085}})

In [197]:
lightgbm_model.

In [179]:
lightgbm_model.feature_importance()

array([52475, 49984, 67309, ...,     0,   368,  2979])

In [180]:
print(lightgbm_model.feature_name())

['Column_0', 'Column_1', 'Column_2', 'Column_3', 'Column_4', 'Column_5', 'Column_6', 'Column_7', 'Column_8', 'Column_9', 'Column_10', 'Column_11', 'Column_12', 'Column_13', 'Column_14', 'Column_15', 'Column_16', 'Column_17', 'Column_18', 'Column_19', 'Column_20', 'Column_21', 'Column_22', 'Column_23', 'Column_24', 'Column_25', 'Column_26', 'Column_27', 'Column_28', 'Column_29', 'Column_30', 'Column_31', 'Column_32', 'Column_33', 'Column_34', 'Column_35', 'Column_36', 'Column_37', 'Column_38', 'Column_39', 'Column_40', 'Column_41', 'Column_42', 'Column_43', 'Column_44', 'Column_45', 'Column_46', 'Column_47', 'Column_48', 'Column_49', 'Column_50', 'Column_51', 'Column_52', 'Column_53', 'Column_54', 'Column_55', 'Column_56', 'Column_57', 'Column_58', 'Column_59', 'Column_60', 'Column_61', 'Column_62', 'Column_63', 'Column_64', 'Column_65', 'Column_66', 'Column_67', 'Column_68', 'Column_69', 'Column_70', 'Column_71', 'Column_72', 'Column_73', 'Column_74', 'Column_75', 'Column_76', 'Column_

In [181]:
lightgbm_model.feature_name = X.columns

In [182]:
lightgbm_model.feature_name

Index(['Min', 'Max', 'Mean', 'Max - Min', 'FLN_count', 'Upc_count', 'DD_count',
       'Sum', 'FLN_DD_Ratio', 'UPc_DD_Ratio',
       ...
       'FLN__9961.0', 'FLN__9966.0', 'FLN__9967.0', 'FLN__9970.0',
       'FLN__9971.0', 'FLN__9974.0', 'FLN__9975.0', 'FLN__9991.0',
       'FLN__9997.0', 'FLN__9998.0'],
      dtype='object', length=5165)

## 모델 저장

In [183]:
joblib.dump(lightgbm_model, './LightGBM/LGB_train_round{}_lr{}.pkl'.format(num_boost_round, learning_rate))

['./LightGBM/LGB_train_round4000_lr0.008.pkl']

In [184]:
lightgbm_model = joblib.load('./LightGBM/LGB_train_round{}_lr{}.pkl'.format(num_boost_round, learning_rate))

In [185]:
lightgbm_model.best_iteration

0

In [186]:
lightgbm_model.save_model('./LightGBM/LGB_train_round{}_lr{}.model'.format(num_boost_round, learning_rate))

### prediction

In [187]:
%time result = lightgbm_model.predict(test)

CPU times: user 1h 19min 15s, sys: 40.4 s, total: 1h 19min 56s
Wall time: 14min 34s


In [188]:
myslack.send_slack('LightGBM prediction finish!')

<Response [200]>


In [189]:
result

array([[1.80736464e-08, 3.59038107e-07, 1.40440506e-05, ...,
        2.65835624e-04, 4.64835506e-05, 1.02367126e-03],
       [3.04990527e-06, 2.31792286e-05, 3.67664032e-04, ...,
        1.77583786e-02, 5.98915099e-04, 8.81473884e-03],
       [3.37168084e-08, 8.94251916e-10, 2.58991160e-07, ...,
        6.19389381e-08, 3.50007359e-08, 9.99931203e-01],
       ...,
       [8.66531498e-06, 1.33576773e-06, 1.63198697e-04, ...,
        1.48127120e-05, 1.97850380e-05, 3.63627198e-03],
       [4.77135945e-09, 4.15136738e-09, 7.52780573e-07, ...,
        5.14139452e-04, 3.27650367e-03, 1.67633006e-05],
       [5.36301611e-08, 4.75592975e-08, 8.83106506e-07, ...,
        1.44357289e-03, 3.50297788e-04, 2.32392725e-05]])

### Make Submission CSV file

In [190]:
samplesub = pd.read_csv('./data/sample_submission.csv')

In [191]:
subform_df_columns = samplesub.columns[1:]
result_df = pd.DataFrame(result)
result_df.columns = subform_df_columns
subform_df = pd.concat([test.reset_index()['VisitNumber'],result_df],axis=1)
subform_df.set_index('VisitNumber',inplace=True)
subform_df.tail()

,TripType_3,TripType_4,TripType_5,TripType_6,TripType_7,TripType_8,TripType_9,TripType_12,TripType_14,TripType_15,TripType_18,TripType_19,...,TripType_34,TripType_35,TripType_36,TripType_37,TripType_38,TripType_39,TripType_40,TripType_41,TripType_42,TripType_43,TripType_44,TripType_999
VisitNumber,,,,,,,,,,,,,,,,,,,,,,,,,
191338,1.987282e-07,4.198027e-07,5.694994e-05,1.573755e-05,4.318863e-04,0.000115,0.000064,0.000018,1.593477e-08,0.000262,9.651993e-06,1.899166e-06,...,6.476241e-06,0.017298,0.000088,0.000294,0.001768,0.260256,0.003863,0.000054,0.000292,0.003737,0.002081,0.000630
191339,3.484471e-07,3.397986e-07,4.717448e-05,1.721401e-05,2.363250e-04,0.000133,0.000406,0.000006,2.376029e-09,0.000176,2.274112e-04,1.286245e-06,...,1.435334e-05,0.002186,0.000052,0.000071,0.000429,0.005685,0.000027,0.000691,0.073210,0.010466,0.001729,0.000234
191340,8.665315e-06,1.335768e-06,1.631987e-04,4.671471e-05,3.273531e-03,0.937880,0.047466,0.000026,3.564912e-09,0.000591,7.928273e-04,8.777285e-07,...,6.724287e-06,0.003616,0.000190,0.000069,0.000258,0.000303,0.000028,0.000009,0.000571,0.000015,0.000020,0.003636
191341,4.771359e-09,4.151367e-09,7.527806e-07,8.803936e-08,6.532289e-07,0.000004,0.000008,0.000002,5.048110e-10,0.000019,1.554709e-07,4.717153e-08,...,1.543882e-07,0.000002,0.000015,0.000003,0.000005,0.011712,0.000083,0.000275,0.000829,0.000514,0.003277,0.000017
191348,5.363016e-08,4.755930e-08,8.831065e-07,1.586243e-07,6.756456e-06,0.000090,0.000040,0.000002,2.955739e-10,0.000006,7.867019e-07,5.631004e-08,...,1.236883e-07,0.000002,0.000002,0.000010,0.000005,0.000114,0.000028,0.006577,0.005780,0.001444,0.000350,0.000023


In [192]:
%time subform_df.to_csv('./LightGBM/LGB_train_round{}_lr{}.csv'.format(num_boost_round, learning_rate))

CPU times: user 5.75 s, sys: 102 ms, total: 5.85 s
Wall time: 5.89 s


In [193]:
myslack.send_slack('Making LightGBM CSV finish!!')

<Response [200]>


---

---

### Ver3

# Modeling 6 - LightGBM

In [13]:
# in test, not in train
not_in_train = [i for i in test.columns if i not in train.columns[1:]]

In [14]:
# in train, not in test
not_in_test = [i for i in train.columns[1:] if i not in test.columns]

In [15]:
len(not_in_train)

328

In [16]:
len(not_in_test)

328

In [17]:
train.drop(columns=not_in_test,inplace=True)

In [21]:
train.shape

(95674, 5166)

In [18]:
test.drop(columns=not_in_train,inplace=True)

In [22]:
test.shape

(95674, 5165)

### Train_X, Train_y

In [72]:
X = train.drop(columns='TripType')
y = train['TripType']

In [73]:
X = csr_matrix(X); X

<95674x5165 sparse matrix of type '<class 'numpy.float64'>'
	with 2214766 stored elements in Compressed Sparse Row format>

In [74]:
csr_test = csr_matrix(test); csr_test

<95674x5165 sparse matrix of type '<class 'numpy.float64'>'
	with 2221183 stored elements in Compressed Sparse Row format>

### LightGBM

In [85]:
%%time

estimator=250
depth=20

lightgbm_model = lightgbm.LGBMClassifier(n_estimators=estimator, max_depth=depth, n_jobs=6).fit(X,y)

myslack_incomming.send_slack('Ver3_lightgbm {},{} modeling finish!'.format(estimator,depth))

<Response [200]>
CPU times: user 14min 16s, sys: 2min 1s, total: 16min 17s
Wall time: 20min 7s


### Confusion Matrix

In [86]:
%time confusion_matrix(y, lightgbm_model.predict(X))

CPU times: user 2min 43s, sys: 235 ms, total: 2min 44s
Wall time: 27.8 s


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


array([[3643,    0,    0, ...,    0,    0,    0],
       [   0,  333,   13, ...,    0,    0,    0],
       [   0,    0, 4456, ...,    1,    0,    1],
       ...,
       [   0,    0,    3, ...,  739,    0,    1],
       [   0,    0,    0, ...,    0, 1130,    0],
       [ 300,    2,   17, ...,    0,    0, 7705]])

### Train Test Split

In [87]:
X_train, X_test, y_train, y_test \
= train_test_split(X, y, test_size=0.2)

X_train.shape[0], X_test.shape[0], len(y_train), len(y_test)

(76539, 19135, 76539, 19135)

In [88]:
%%time
y_pred = lightgbm_model.predict(X_test)

CPU times: user 32.1 s, sys: 40.9 ms, total: 32.1 s
Wall time: 5.44 s


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [89]:
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          3       0.92      1.00      0.96       724
          4       1.00      0.98      0.99        62
          5       0.93      0.97      0.95       950
          6       0.99      1.00      0.99       266
          7       0.90      0.92      0.91      1132
          8       0.93      0.94      0.94      2438
          9       0.87      0.93      0.90      1876
         12       1.00      1.00      1.00        45
         14       1.00      1.00      1.00         1
         15       0.93      0.93      0.93       172
         18       0.97      0.89      0.93       103
         19       0.97      0.94      0.96        83
         20       1.00      0.96      0.98       135
         21       0.99      0.98      0.99       143
         22       0.90      0.90      0.90       176
         23       0.96      0.96      0.96        25
         24       0.93      0.91      0.92       509
         25       0.92      0.95      0.93   

### prediction

In [90]:
%%time
result = lightgbm_model.predict_proba(csr_test)

CPU times: user 2min 38s, sys: 158 ms, total: 2min 38s
Wall time: 26.5 s


### Make Submission CSV file

In [91]:
samsub = pd.read_csv('./data/sample_submission.csv')
subform_df_columns = samsub.columns[1:]
result_df = pd.DataFrame(result)
result_df.columns = subform_df_columns
subform_df = pd.concat([test.reset_index()['VisitNumber'],result_df],axis=1)
subform_df.set_index('VisitNumber',inplace=True)
subform_df.tail()

,TripType_3,TripType_4,TripType_5,TripType_6,TripType_7,TripType_8,TripType_9,TripType_12,TripType_14,TripType_15,TripType_18,TripType_19,...,TripType_34,TripType_35,TripType_36,TripType_37,TripType_38,TripType_39,TripType_40,TripType_41,TripType_42,TripType_43,TripType_44,TripType_999
VisitNumber,,,,,,,,,,,,,,,,,,,,,,,,,
191338,1.653469e-06,1.708501e-06,0.000095,6.576896e-05,0.000420,0.000082,0.000105,0.000032,7.153566e-08,0.000212,1.555688e-05,3.733026e-06,...,3.804306e-06,0.028980,0.000074,0.000188,0.002576,0.174151,0.003168,0.000104,0.000413,0.002624,0.001734,0.000390
191339,3.016177e-06,2.037850e-06,0.000147,3.706446e-05,0.000541,0.000344,0.001003,0.000031,6.757531e-08,0.000379,7.368699e-04,6.861530e-06,...,2.131447e-05,0.004767,0.000123,0.000288,0.000965,0.011179,0.000135,0.001577,0.150306,0.023044,0.003968,0.000647
191340,2.779387e-05,3.239346e-06,0.000224,1.055917e-04,0.004107,0.923192,0.055008,0.000101,7.873830e-08,0.001107,9.211155e-04,7.017644e-06,...,1.258328e-05,0.004283,0.000240,0.000122,0.000415,0.000410,0.000055,0.000027,0.000694,0.000034,0.000038,0.007299
191341,4.918564e-08,3.459018e-08,0.000004,1.994391e-07,0.000004,0.000017,0.000008,0.000006,5.188286e-09,0.000022,9.587239e-07,2.126053e-07,...,2.430074e-07,0.000006,0.000012,0.000007,0.000010,0.021022,0.000203,0.000345,0.002326,0.000816,0.005027,0.000030
191348,9.904227e-08,4.797844e-08,0.000002,4.521200e-07,0.000008,0.000881,0.000040,0.000004,3.440554e-09,0.000006,1.095652e-06,1.753617e-07,...,2.243617e-07,0.000004,0.000005,0.000006,0.000005,0.000168,0.000026,0.002551,0.005463,0.001173,0.000387,0.000022


In [92]:
subform_df.to_csv('./LightGBM/Ver3_lgbm_est{}_dep{}.csv'.format(estimator,depth))

In [93]:
myslack_incomming.send_slack('Making Ver3_lightgbm_est{},dep{} to CSV finish!!'.format(estimator,depth))

<Response [200]>


---

## 모델 저장

In [24]:
from sklearn.externals import joblib

joblib.dump(lightgbm_model, 'Ver3_lightgbm_est{},dep{}.pkl'.format(estimator,depth)) 
my_lightgbm = joblib.load('Ver3_lightgbm_est{},dep{}.pkl'.format(estimator,depth)) 